In [ ]:
# bzq modifying
len_x_target = 4
len_y_target = 4
stride_x_target = 1
stride_y_target = 1

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 60
pixels = 0

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d

In [ ]:
# 劃出曲線，預測mnist

#原始資料集的training data用沒有 mask 的模型做訓練得到weights A
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Define LeNet model
def create_lenet():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    return model

# Define MLP model
def create_mlp():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    return model

# Compile and train model
def train_model(model, x_train, y_train, x_val, y_val, epochs=20, batch_size=128):
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), callbacks=[early_stopping])

# Split data for validation
x_val, y_val = x_train[-10000:], y_train[-10000:]
x_train, y_train = x_train[:-10000], y_train[:-10000]

# Train LeNet model
lenet_model = create_lenet()
if os.path.exists("lenet.weights.h5"):
    lenet_model.load_weights("lenet.weights.h5")
else:
    train_model(lenet_model, x_train, y_train, x_val, y_val)

# Train MLP model
#mlp_model = create_mlp()
#train_model(mlp_model, x_train, y_train, x_val, y_val)

# Ensemble predictions
def ensemble_predictions(models, x_test):
    predictions = [model.predict(x_test) for model in models]
    return np.mean(predictions, axis=0)

# Create ensemble of LeNet models
#lenet_ensemble = [create_lenet() for _ in range(10)]
#for model in lenet_ensemble:
#    train_model(model, x_train, y_train, x_val, y_val)

# Get ensemble predictions
#ensemble_preds = ensemble_predictions(lenet_ensemble, x_test)

# Save weights for LeNet model
lenet_model.save_weights('lenet.weights.h5')

# Save weights for MLP model
#mlp_model.save_weights('mlp.weights.h5')

# Save weights for each model in the ensemble
#for i, model in enumerate(lenet_ensemble):
#    model.save_weights(f'lenet_ensemble.weights_{i}.h5')



In [ ]:
# 得到了bzq 正確的函數，拿來做mnist 正確的預測
import numpy as np
import tensorflow as tf
from collections import Counter
import scipy.ndimage

def single_data_rotate_roll_preprocessing(original_data, angle, pixels):
    new_data = np.copy(original_data).astype(np.float32)
    #new_data = np.expand_dims(new_data, axis=-1)
    new_data = scipy.ndimage.rotate(new_data, angle, reshape=False)
    new_data = np.expand_dims(new_data, axis=0)  # 增加批次維度
    new_data = np.roll(new_data, pixels, axis=-1)
    return new_data

def single_data_bzq_mask_preprocessing_mnist(original_data, start_x, start_y, len_x, len_y, magnification):
    new_data = np.copy(original_data)
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    new_data[start_y:start_y + len_y, start_x:start_x + len_x] = 1
    return new_data

lenet_model = create_lenet()
lenet_model.load_weights('lenet.weights.h5')

bzq = []

correct_predictions_mnist = []
incorrect_predictions_mnist = []

x_test = np.array(x_test)

# 預先計算旋轉和滾動的資料
preprocessed_data = np.array([single_data_rotate_roll_preprocessing(x_test[k], angle, pixels) for k in range(10000)]).reshape(-1, 28, 28, 1)

# 顯示影像
for i in range(2):
    plt.imshow(preprocessed_data[i, :, :, 0], cmap='gray')
plt.show()

original_predictions_mnist = lenet_model.predict(preprocessed_data, verbose=0)
for i in range(10000):
    if np.argmax(original_predictions_mnist[i]) == np.argmax(y_test[i]):
        correct_predictions_mnist.append(i)
    else:
        incorrect_predictions_mnist.append(i)

# bzq modifying
len_x = len_x_target
len_y = len_y_target
stride_x = stride_x_target
stride_y = stride_y_target

for k in range(len(x_test)):
    single_data_bzq_classification_record = []
    targets = []
    for i in range(0, 28 - len_y, stride_y):
        for j in range(0, 28 - len_x, stride_x):
            target = single_data_bzq_mask_preprocessing_mnist(preprocessed_data[k], i, j, len_x, len_y, 0)
            targets.append(target)

    # 批次預測
    predictions = lenet_model.predict(np.vstack(targets).reshape(-1, 28, 28, 1), verbose=0)
    max_bzq_indices = np.argmax(predictions, axis=1)
    
    single_data_bzq_classification_record.extend(max_bzq_indices)
    
    counter = Counter(single_data_bzq_classification_record)
    most_common_num, most_common_count = counter.most_common(1)[0]
    
    bzq.append((len(single_data_bzq_classification_record) - most_common_count) / len(single_data_bzq_classification_record))
    #bzq_acc.append(np.sum(single_data_bzq_classification_record == np.argmax(y_test[k])) / len(single_data_bzq_classification_record))
    #bzq_conf.append(most_common_count / len(single_data_bzq_classification_record))

    original_data = single_data_bzq_mask_preprocessing_mnist(preprocessed_data[k], 0, 0, 0, 0, 0)
    original_prediction = lenet_model.predict(original_data.reshape(-1, 28, 28, 1), verbose=0)

    max_original_index = np.argmax(original_prediction)



In [ ]:
print(len(preprocessed_data))
print(f"{len(correct_predictions_mnist)}, {len(incorrect_predictions_mnist)}")
#print(bzq)
bzq_mnist = np.array(bzq)
result_bzq_mnist = 1 / bzq_mnist

'''for i, val in enumerate(result_bzq_mnist):
    if val > 1:
        result_bzq_mnist[i] = 1
    elif val < 0:
        result_bzq_mnist[i] = 0'''
        
#print(result_bzq_mnist)

counts, bins, patches = plt.hist(bzq_mnist, bins=bins_size)
plt.title('Cumulative Histogram of Correct Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置
plt.show()

# 打印結果
plt.boxplot(bzq_mnist)
plt.show()

# 繪製點狀圖
plt.scatter(bzq, result_bzq_mnist)

# 設定標題和軸標籤
plt.title('Scatter Plot of x vs f')
plt.xlabel('x')
plt.ylabel('f')

# 顯示圖表
plt.show()

bzq_correct_mnist = np.array([bzq_mnist[i] for i in correct_predictions_mnist])
bzq_incorrect_mnist = np.array([bzq_mnist[i] for i in incorrect_predictions_mnist])

result_bzq_correct_mnist = np.array([result_bzq_mnist[i] for i in correct_predictions_mnist])
result_bzq_incorrect_mnist = np.array([result_bzq_mnist[i] for i in incorrect_predictions_mnist])

# 打印結果

counts, bins, patches = plt.hist(bzq_correct_mnist, bins=bins_size)
plt.title('Cumulative Histogram of Correct Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置
plt.show()
plt.boxplot(bzq_correct_mnist)
plt.show()
# 繪製點狀圖
plt.scatter(bzq_correct_mnist, result_bzq_correct_mnist)

# 設定標題和軸標籤
plt.title('Scatter Plot of x vs f')
plt.xlabel('x')
plt.ylabel('f')

# 顯示圖表
plt.show()

# 打印結果
counts, bins, patches = plt.hist(bzq_incorrect_mnist, bins=bins_size)
plt.title('Cumulative Histogram of Incorrect Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置
plt.show()
plt.boxplot(bzq_incorrect_mnist)
plt.show()
# 繪製點狀圖
plt.scatter(bzq_incorrect_mnist, result_bzq_incorrect_mnist)

# 設定標題和軸標籤
plt.title('Scatter Plot of x vs f')
plt.xlabel('x')
plt.ylabel('f')

# 顯示圖表
plt.show()

In [ ]:
# 劃出confidence-acc 圖: confidence由bzq提供，acc由該confidence數值底下預測準確的

result_pred_mnist = np.ones(10000) 
for i in incorrect_predictions_mnist:
    result_pred_mnist[i] = 0

print(sum(result_pred_mnist))

result_mnist_dict = {}
for i, val in enumerate(result_bzq_mnist):
    if val not in result_mnist_dict.keys():
        result_mnist_dict[val] = [result_pred_mnist[i]]
    else:
        result_mnist_dict[val].append(result_pred_mnist[i])

# 根據key進行排序
#result_mnist_dict = dict(sorted(result_mnist_dict.items(), reverse=True))

#print(result_mnist_dict)

# 初始化信心值和準確率列表
confidence_values = []
accuracies = []
element_counts = []

# 計算每個信心值範圍的準確率
for confidence in sorted(result_mnist_dict.keys(), reverse=True):
    combined_results = []
    for key in result_mnist_dict:
        if key >= confidence:
            combined_results.extend(result_mnist_dict[key])
    element_count = len(combined_results)
    accuracy = np.mean(combined_results)
    confidence_values.append(confidence)
    accuracies.append(accuracy)
    element_counts.append(element_count)

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values, accuracies, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy (Rotated 60°)')
plt.grid(True)
plt.show()

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values, element_counts, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Number of Elements (p(y|x) >= τ)')
plt.title('Confidence Threshold vs Number of Elements')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

confidence_values_scaled = np.array(confidence_values)
confidence_values_scaled = 2 / np.pi * np.arctan(confidence_values_scaled)
#confidence_values_scaled = confidence_values_scaled * confidence_values_scaled / (1 - confidence_values_scaled * confidence_values_scaled)
                                                                                  

#print(confidence_values_scaled)

'''# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_scaled, accuracies, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy (Rotated 60°)')
plt.grid(True)
plt.show()'''

scaler = MinMaxScaler()
confidence_values_scaled = scaler.fit_transform(np.array(confidence_values_scaled).reshape(-1, 1)).flatten()
#print(confidence_values_scaled)

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_scaled, accuracies, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy (Rotated 60°)')
plt.grid(True)
plt.show()

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_scaled, element_counts, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Number of Elements (p(y|x) >= τ)')
plt.title('Confidence Threshold vs Number of Elements')
plt.grid(True)
plt.show()
